In [1]:
import pandas as pd
import numpy as np

In [87]:
df_resale = pd.read_csv('../data/cleaned/resale_price_cleaned.csv')

### Add inflation-adjusted resale price
Real Price = Nominal Price * (CPI Base Year/ CPI Curret Month)

In [6]:
df_deflated_resale = df_resale.copy()
cpi_base = 100
df_deflated_resale["deflated_resale_price"] = df_deflated_resale["resale_price"] * (100/ df_deflated_resale["CPI (base 2024-12)"])

### Year, Month

In [88]:
df_resale['month'] = pd.to_datetime(df_resale['month'])
df_resale['year'] = df_resale['month'].dt.year
df_resale['month'] = df_resale['month'].dt.month

### Drop correlated variables

In [89]:
df_resale.dropna(inplace = True)

In [90]:
df_resale.drop(columns = ['flat_type', 'building_age_2025', 'total_unemployment_rate', 'flat_model',
                                   'CPI (base 2024-12)'], inplace = True)

In [79]:
df_resale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196986 entries, 0 to 201058
Data columns (total 21 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   month                       196986 non-null  int64  
 1   town                        196986 non-null  object 
 2   storey_range                196986 non-null  int64  
 3   floor_area_sqm              196986 non-null  float64
 4   remaining_lease             196986 non-null  float64
 5   resale_price                196986 non-null  float64
 6   lat                         196986 non-null  float64
 7   lon                         196986 non-null  float64
 8   nearest_mrt_distance        196986 non-null  float64
 9   nearest_bus_distance        196986 non-null  float64
 10  education_score             196986 non-null  float64
 11  shopping_score              196986 non-null  float64
 12  food_score                  196986 non-null  float64
 13  recreation_sco

In [9]:
import os

current_directory = os.getcwd()
print(current_directory)

/Users/judy/Desktop/DSE3101-Project/DSE3101-HDB-Decode/code


### Add ethnicity and religion columns 

In [91]:
df_2 = pd.read_csv("../data/cleaned/resale_price_normalized_demo.csv")

In [69]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196986 entries, 0 to 196985
Data columns (total 38 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         196986 non-null  object 
 1   town                          196986 non-null  object 
 2   flat_type                     196986 non-null  float64
 3   storey_range                  196986 non-null  float64
 4   floor_area_sqm                196986 non-null  float64
 5   flat_model                    196986 non-null  object 
 6   remaining_lease               196986 non-null  float64
 7   resale_price                  196986 non-null  float64
 8   lat                           196986 non-null  float64
 9   lon                           196986 non-null  float64
 10  nearest_mrt_distance          196986 non-null  float64
 11  nearest_bus_distance          196986 non-null  float64
 12  education_score               196986 non-nul

In [92]:
# from another dataset


# drop NoReligion and Others
df_2 = df_2[['town','NoReligion','Buddhism','Taoism1','Islam','Hinduism','Sikhism',
            'Christianity_Catholic','Christianity_OtherChristians','OtherReligions','Chinese', 'Malays','Indians','Others']]

df_2_selected = df_2[['town','Buddhism']]

In [93]:
df_resale = df_resale.sort_values('town').reset_index(drop=True)
df_2_sorted = df_2.sort_values('town').reset_index(drop=True)
(df_resale['town'] == df_2_sorted['town']).all()

True

In [94]:
columns_to_add = ['NoReligion','Buddhism','Taoism1','Islam','Hinduism','Sikhism','Christianity_Catholic',
                  'Christianity_OtherChristians','OtherReligions','Chinese', 'Malays','Indians','Others']

for col in columns_to_add:
    df_resale[col] = df_2_sorted[col]

Merge takes up too much RAM memory...

In [24]:
df_deflated_resale_small = df_deflated_resale.head(2000)
df_2_small = df_2.head(2000)
df_com = pd.merge(df_deflated_resale_small, df_2_small, on = 'town', how = 'left')

In [ ]:
df_com = pd.merge(df_deflated_resale, df_2_selected, on = 'town', how = 'left')

: 

In [81]:
df_resale.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196986 entries, 0 to 201058
Data columns (total 34 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         196986 non-null  int64  
 1   town                          196986 non-null  object 
 2   storey_range                  196986 non-null  int64  
 3   floor_area_sqm                196986 non-null  float64
 4   remaining_lease               196986 non-null  float64
 5   resale_price                  196986 non-null  float64
 6   lat                           196986 non-null  float64
 7   lon                           196986 non-null  float64
 8   nearest_mrt_distance          196986 non-null  float64
 9   nearest_bus_distance          196986 non-null  float64
 10  education_score               196986 non-null  float64
 11  shopping_score                196986 non-null  float64
 12  food_score                    196986 non-nul

### Standardization (z-score)

In [95]:
df_resale_standardized = df_resale.copy()
def standardized(col):
    return (col - col.mean()) / col.std()

columns_to_standardized = df_resale.select_dtypes(include=['float64', 'int64']).columns.difference(['year','month','lat','lon'])
df_resale_standardized[columns_to_standardized] = df_resale[columns_to_standardized].apply(standardized)
df_resale_standardized.head()


,month,town,storey_range,floor_area_sqm,remaining_lease,resale_price,lat,lon,nearest_mrt_distance,nearest_bus_distance,...,Islam,Hinduism,Sikhism,Christianity_Catholic,Christianity_OtherChristians,OtherReligions,Chinese,Malays,Indians,Others
0,1,ANG MO KIO,0.378198,-0.208431,-1.426660,-0.151534,1.370561,103.854823,-0.151327,0.372923,...,-1.422367,-0.272024,-0.715458,1.007616,0.102796,-1.198364,5.732743e-15,-1.608913e-15,-1.121358e-14,6.562959e-15
1,5,ANG MO KIO,0.378198,-0.624584,-0.976570,-0.767368,1.362758,103.858015,1.407387,1.422031,...,-1.422367,-0.272024,-0.715458,1.007616,0.102796,-1.198364,5.732743e-15,-1.608913e-15,-1.121358e-14,6.562959e-15
2,7,ANG MO KIO,0.883747,2.122029,-0.982258,2.269221,1.362758,103.858015,1.407387,1.422031,...,-1.422367,-0.272024,-0.715458,1.007616,0.102796,-1.198364,5.732743e-15,-1.608913e-15,-1.121358e-14,6.562959e-15
3,8,ANG MO KIO,0.378198,-1.248814,-0.988657,-0.995611,1.362758,103.858015,1.407387,1.422031,...,-1.422367,-0.272024,-0.715458,1.007616,0.102796,-1.198364,5.732743e-15,-1.608913e-15,-1.121358e-14,6.562959e-15
4,1,ANG MO KIO,-0.632900,-1.248814,-1.017810,-1.085646,1.362758,103.858015,1.407387,1.422031,...,-1.422367,-0.272024,-0.715458,1.007616,0.102796,-1.198364,5.732743e-15,-1.608913e-15,-1.121358e-14,6.562959e-15


### One-Hot encoding

In [96]:
# drop the first value to avoid dummy variable trap
df_encoded = pd.get_dummies(df_resale_standardized, columns=['town'], drop_first=True)

In [100]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196986 entries, 0 to 196985
Data columns (total 58 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   month                         196986 non-null  int64  
 1   storey_range                  196986 non-null  float64
 2   floor_area_sqm                196986 non-null  float64
 3   remaining_lease               196986 non-null  float64
 4   resale_price                  196986 non-null  float64
 5   lat                           196986 non-null  float64
 6   lon                           196986 non-null  float64
 7   nearest_mrt_distance          196986 non-null  float64
 8   nearest_bus_distance          196986 non-null  float64
 9   education_score               196986 non-null  float64
 10  shopping_score                196986 non-null  float64
 11  food_score                    196986 non-null  float64
 12  recreation_score              196986 non-nul

In [97]:
df_encoded.to_csv('../data/cleaned/standardized_encoded_delfated_resale_price.csv', index=False)